# Use Cases

We will use [Rotten Tomatoes](https://editorial.rottentomatoes.com/) webpage, which is an American review-aggregation website for film and television. We will use `soupsavvy`to extract information from there.

In [1]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "en-US,en;q=0.9",
}
url = "https://editorial.rottentomatoes.com/guide/best-movies-of-all-time/"
response = requests.get(url, headers=headers)
bs = BeautifulSoup(response.text, "lxml")

In [ ]:
from soupsavvy import ClassSelector, TypeSelector

movie_selector = ClassSelector("movie") & TypeSelector("p")
result = movie_selector.find_all(bs)

print(f"Found {len(result)} movies")
result[0]

In [ ]:
from soupsavvy.operations import Text
from soupsavvy import ClassSelector

title_selector = (movie_selector >> ClassSelector("title")) | Text(strip=True)
result = title_selector.find_all(bs)

print(f"Found {len(result)} titles")
print(result[:3])

In [ ]:
from soupsavvy import ClassSelector, TypeSelector
from soupsavvy.models import BaseModel, post
from soupsavvy.operations import Text


class Movie(BaseModel):
    __scope__ = (ClassSelector("movie") & TypeSelector("p")) << TypeSelector("tr")

    title = ClassSelector("title") | Text(strip=True)
    score = ClassSelector("score") | Text()
    
    @post("score")
    def process_score(self, score: str) -> int:
        return int(score.strip("%"))


Movie.find(bs)

In [ ]:
from soupsavvy import ClassSelector, TypeSelector
from soupsavvy.models import BaseModel, post
from soupsavvy.operations import Href, Operation, Text
from soupsavvy.selectors.css import FirstOfType


class MovieDetails(BaseModel):
    __scope__ = ClassSelector("details")

    title = ClassSelector("title") | Text(strip=True)
    year = (
        ClassSelector("year")
        | Text()
        | Operation(lambda x: x.strip("()"))
        | Operation(int)
    )
    link = ClassSelector("title") | Href()


class Movie(BaseModel):
    __scope__ = (ClassSelector("movie") & TypeSelector("p")) << TypeSelector("tr")

    rank = (FirstOfType() & TypeSelector("td")) | Text()
    score = ClassSelector("score") | Text()
    details = MovieDetails

    @post("score")
    def process_score(self, score: str) -> int:
        return int(score.strip("%"))

    def __post_init__(self) -> None:
        self.rank = int(str(self.rank).strip("."))


result = Movie.find_all(bs)
print(f"Found {len(result)} movies")

result[0]